#### This notebook is for explorer options to optimization procedures over budget consumption. 

In [1]:
import sys, os 
prj_path = os.path.dirname(os.getcwd())
utils_path = prj_path + '/utils'
if  not utils_path in sys.path:
    print('adding utils to path ')
    sys.path.insert(1, utils_path)

adding utils to path 


In [25]:
from utilities import *
import preprocessing as pre
from google_cloud import BigQuery, Storage
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

import os
import sys

import boto3
import base64

#Cliente BigQuery
AUTH_BIGQUERY = base64.b64decode(os.environ['SECRET_AUTH_BIGQUERY_MODEL'])
bq = BigQuery(AUTH_BIGQUERY)
storage =Storage(AUTH_BIGQUERY)

#Load config.
files = glob.glob(f"{utils_path}/*.yml") 
conf = load_conf(files)

In [ ]:
#Ver como varia el budget de una campaña.
#Ver como varian los incrementales.
#Ver como ranquea el modelo y comparar con los incrementales reales.
#Ver si hay componenete estacional.

#### Budget por campaña

In [16]:
%%time
buget_variation = bq.execute_response(""" 
SELECT 
    push.CAMPAIGN_ID,
    budget.MKT_TOOL_ID,
    budget.BUDGET_LC,
    min(push.SENT_DATE) minDate_push_event,
    max(push.SENT_DATE) maxDate_push_event,
    min(budget.TIM_MONTH) minDate_budget
FROM meli-marketing.MKTPUBLIC.V_BT_PUSH_NOTIFICATION_EVENT push 
LEFT JOIN meli-marketing.PUSH.LK_PUSH_BUDGET_COUPON budget 
ON push.CAMPAIGN_ID = budget.PUSH_CAMPAIGN_ID
AND budget.SIT_SITE_ID = 'MLB'
WHERE 1=1
AND BATCH_ID LIKE 'MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS%'
AND push.SENT_DATE BETWEEN DATE '2022-01-01' AND DATE '2022-03-30'
GROUP BY 1,2,3
ORDEr BY 2,3""")

CPU times: user 32.7 ms, sys: 0 ns, total: 32.7 ms
Wall time: 1.02 s


In [17]:
buget_variation.head(4)

,CAMPAIGN_ID,MKT_TOOL_ID,BUDGET_LC,minDate_push_event,maxDate_push_event,minDate_budget
0,MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS-34,10045401,750000.0,2022-01-05,2022-03-30,2022-01-02
1,MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS-34,10046542,750000.0,2022-01-05,2022-03-30,2022-01-12
2,MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS-34,10166514,254837.0,2022-01-05,2022-03-30,2021-03-01


#### Push events and model prediction.

In [26]:
#model
storage.downlad_file(gs_source_path = 'gs://marketing-modelling/ML/NB/MLB/model/model_MLB.pkl', 
                     destination_file = 'old_model_nb.pkl')
model = pd.read_pickle('old_model_nb.pkl')

In [40]:
%%time
q = pre.fun_query_apply('2022-01-05',PAIS ='MLB')
df = bq.execute_response(q)

CPU times: user 9min 8s, sys: 10.5 s, total: 9min 19s
Wall time: 1h 27min 34s


In [41]:
df.head()

,CUS_CUST_ID,SIT_SITE_ID,CUS_RU_SINCE,TYPE_APP,PROCESS_DATE,TOTAL_PAYMENTS_AMT_SUM,WALLET_PAYMENTS_AMT_SUM,AGGREGATOR_PAYMENTS_AMT_SUM,TOTAL_PAYMENTS_COUNT,CAT_CATEG_ID_COUNT,...,MAX_PRICE_ITEM_F7D,AVG_PRICE_ITEM_F7D,LOG_DATE_COUNT_F7D,OS_NAME,APP_ML,APP_MP,MAX_INSTALL_ML,MIN_INSTALL_ML,MAX_INSTALL_MP,MIN_INSTALL_MP
0,791404320,MLB,2021-12-04,App,2022-01-05,17.232569292,None,17.232569292,1.0,1.0,...,110.610000000,104.077500000,1.0,android,1.0,0.0,2021-12-04,2021-12-04,None,None
1,1025991843,MLB,2021-11-24,Sin App,2022-01-05,None,None,None,NaN,2.0,...,89.970000000,63.480000000,1.0,None,NaN,NaN,None,None,None,None
2,1014912501,MLB,2021-11-08,App,2022-01-05,None,None,None,NaN,5.0,...,1099.000000000,308.954166667,3.0,android,1.0,0.0,2021-11-08,2021-11-08,None,None
3,1017673263,MLB,2021-11-12,App,2022-01-05,None,None,None,NaN,6.0,...,149.900000000,41.431944445,2.0,None,NaN,NaN,None,None,None,None
4,1025713305,MLB,2021-11-24,Sin App,2022-01-05,None,None,None,NaN,1.0,...,256.400000000,256.400000000,1.0,None,NaN,NaN,None,None,None,None


In [ ]:
df['score'] = model.predict_proba(df)

In [18]:
%%time
site = 'MLB'
date_intial = '2022-01-01'
date_end = '2022-03-30'
push_population = bq.execute_response(f"""
    WITH push as ( 
            SELECT 
                push.CUS_CUST_ID,
                push.SIT_SITE_ID,
                push.CAMPAIGN_ID,
                push.SENT_DATE, 
                push.EVENT_TYPE 
            FROM meli-marketing.MKTPUBLIC.V_BT_PUSH_NOTIFICATION_EVENT push 
            WHERE BATCH_ID LIKE 'MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS%' 
                AND push.EVENT_TYPE in ('sent','control') 
                AND push.APP = 'mercadolibre'
                AND push.sit_site_id = '{site}' 
                AND push.SENT_DATE >= DATE '{date_intial}' AND push.SENT_DATE <= DATE '{date_end}' 
             ),
    orders_ as (
        SELECT 
            push.CUS_CUST_ID, 
            push.CAMPAIGN_ID,
            push.SENT_DATE, 
            push.EVENT_TYPE,
            orders.ORD_CREATED_DT,
            ROW_NUMBER() OVER (PARTITION BY push.CUS_CUST_ID ORDER BY orders.ORD_CREATED_DT) AS ORDERING_PURCHASE
        FROM push LEFT JOIN meli-bi-data.WHOWNER.BT_ORD_ORDERS orders
        ON push.CUS_CUST_ID = orders.ORD_BUYER.ID 
            AND push.sit_site_id = orders.SIT_SITE_ID 
            AND ORD_CLOSED_DT is not null
            AND orders.ord_status = 'paid' 
            AND orders.ORD_CLOSED_DT BETWEEN push.SENT_DATE AND push.SENT_DATE + 4 
            AND orders.ORD_CREATED_DT = push.SENT_DATE
        )

    SELECT 
        CUS_CUST_ID,
        CAMPAIGN_ID,
        SENT_DATE,
        EVENT_TYPE,
        CASE WHEN ORD_CREATED_DT is null THEN 0 ELSE 1 END AS CONVERSION
    FROM orders_
    WHERE ORDERING_PURCHASE = 1
    """ 
                                     )

CPU times: user 25 s, sys: 1.32 s, total: 26.3 s
Wall time: 1min 35s


In [19]:
push_population.head(3)

,CUS_CUST_ID,CAMPAIGN_ID,SENT_DATE,EVENT_TYPE,CONVERSION
0,257416060,MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS-34,2022-01-28,control,0
1,325136765,MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS-34,2022-02-03,sent,0
2,466504999,MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS-34,2022-02-05,sent,0


In [21]:
#model
storage.downlad_file(gs_source_path = 'gs://marketing-modelling/ML/NB/MLB/model/model_MLB.pkl', 
                     destination_file = 'old_model_nb.pkl')
model = pd.read_pickle('old_model_nb.pkl')

In [22]:
model = pd.read_pickle('old_model_nb.pkl')

In [23]:
model.predict_proba()

In [53]:
a = 0 
B=100
delta = 12
for i in np.arange(2,30,1):
    a += 1/(i-1)

In [54]:
(B-delta)*a

345.59105142904036